<a href="https://colab.research.google.com/github/alvredo5/data_mining/blob/main/Adaboost_G231220071.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Load libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
#import train_test_split function
from sklearn.model_selection import train_test_split
#import scikit-learn matrics module for accurary calculation
from sklearn import metrics

In [13]:
#load data
iris = datasets.load_iris()
X = iris
y = iris.target

In [14]:
#load data
iris = datasets.load_iris()
X = iris.data  # Use iris.data for features
y = iris.target

In [15]:
#split dataset into training set and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
# 70% training and 30% test

In [16]:
# Create adaboost classifier object
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
# Train Adaboost Classifier
model = abc.fit(X_train, y_train)
# predict the  response for dataset
y_pred = model.predict(X_test)

In [17]:
# Model Accurary, how often is the classifier correct?
print("Accurary:",metrics.accuracy_score(y_test, y_pred))

Accurary: 0.9777777777777777


In [18]:
!git clone https://github.com/eriklindernoren/ML-From-Scratch

fatal: destination path 'ML-From-Scratch' already exists and is not an empty directory.


In [19]:
import sys
sys.path.append('/content/ML-Scratch')

In [20]:
data = datasets.load_digits()

In [22]:
# Error "No module named 'mlfromscratch'" menunjukkan bahwa modul
# mlfromscratch tidak tersedia di lingkungan Python Anda. Modul ini
# bukan bagian dari pustaka standar Python dan perlu diinstal secara
# terpisah. Jika modul ini tidak tersedia, kita bisa membuat fungsi
# bantu (helper functions) yang diperlukan dalam skrip yang sama.

from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd

# Fungsi bantu untuk membagi data menjadi set pelatihan dan pengujian
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:  # Jika diberikan, gunakan seed untuk hasil yang konsisten
        np.random.seed(random_state)
    indices = np.arange(X.shape[0])  # Buat array indeks data
    np.random.shuffle(indices)  # Acak indeks
    test_size = int(len(indices) * test_size)  # Tentukan ukuran set pengujian
    test_indices = indices[:test_size]  # Indeks untuk set pengujian
    train_indices = indices[test_size:]  # Indeks untuk set pelatihan
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# Fungsi bantu untuk menghitung akurasi prediksi
def accuracy_score(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)  # Hitung akurasi
    return accuracy

# Kelas untuk plotting boundary keputusan
class Plot:
    @staticmethod
    def plot_decision_boundary(clf, X, y):
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                             np.arange(y_min, y_max, 0.01))
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        plt.contourf(xx, yy, Z, alpha=0.8)
        plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k')
        plt.show()

# Decision stump sebagai pengklasifikasi lemah untuk Adaboost
class DecisionStump():
    def __init__(self):
        self.polarity = 1  # Polaritas untuk menentukan kelas
        self.feature_index = None  # Indeks fitur yang digunakan untuk klasifikasi
        self.threshold = None  # Nilai threshold untuk fitur
        self.alpha = None  # Nilai alpha yang menunjukkan kekuatan pengklasifikasi

# Adaboost
class Adaboost():
    """Boosting method yang menggunakan sejumlah pengklasifikasi lemah
    dalam ensemble untuk membuat pengklasifikasi kuat. Implementasi ini
    menggunakan decision stumps, yang merupakan Decision Tree satu level.

    Parameters:
    -----------
    n_clf: int
        Jumlah pengklasifikasi lemah yang akan digunakan.
    """
    def __init__(self, n_clf=5):
        self.n_clf = n_clf  # Set jumlah pengklasifikasi lemah

    def fit(self, X, y):
        n_samples, n_features = np.shape(X)  # Dapatkan jumlah sampel dan fitur
        w = np.full(n_samples, (1 / n_samples))  # Inisialisasi bobot sampel
        self.clfs = []  # List untuk menyimpan pengklasifikasi

        for _ in range(self.n_clf):  # Untuk setiap pengklasifikasi lemah
            clf = DecisionStump()  # Inisialisasi decision stump
            min_error = float('inf')  # Inisialisasi error minimum

            for feature_i in range(n_features):  # Untuk setiap fitur
                feature_values = np.expand_dims(X[:, feature_i], axis=1)  # Ekspansi dimensi fitur
                unique_values = np.unique(feature_values)  # Dapatkan nilai unik fitur

                for threshold in unique_values:  # Untuk setiap nilai threshold
                    p = 1
                    prediction = np.ones(n_samples)  # Prediksi awal semua 1
                    prediction[X[:, feature_i] < threshold] = -1  # Ubah prediksi jika kurang dari threshold

                    error = sum(w[y != prediction])  # Hitung error

                    if error > 0.5:  # Jika error lebih dari 0.5, ubah polaritas
                        error = 1 - error
                        p = -1

                    if error < min_error:  # Simpan pengaturan terbaik
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_index = feature_i
                        min_error = error

            EPS = 1e-10  # Nilai epsilon kecil untuk mencegah pembagian oleh nol
            clf.alpha = 0.5 * np.log((1.0 - min_error + EPS) / (min_error + EPS))  # Hitung alpha
            predictions = np.ones(n_samples)  # Prediksi awal semua 1
            negative_idx = (clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold)
            predictions[negative_idx] = -1  # Ubah prediksi jika kurang dari threshold

            w *= np.exp(-clf.alpha * y * predictions)  # Perbarui bobot
            w /= np.sum(w)  # Normalisasi bobot
            self.clfs.append(clf)  # Tambahkan pengklasifikasi ke list

    def predict(self, X):
        clf_preds = [clf.alpha * (clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold).astype(int)
                     for clf in self.clfs]  # Kumpulkan prediksi dari semua pengklasifikasi
        y_pred = np.sum(clf_preds, axis=0)  # Jumlahkan prediksi
        y_pred = np.sign(y_pred)  # Ambil tanda dari hasil penjumlahan
        return y_pred

# Contoh penggunaan
if __name__ == "__main__":
    data = datasets.load_breast_cancer()  # Load dataset kanker payudara
    X = data.data  # Fitur
    y = data.target  # Label
    y[y == 0] = -1  # Ubah label 0 menjadi -1

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)  # Bagi data menjadi set pelatihan dan pengujian

    clf = Adaboost(n_clf=5)  # Inisialisasi Adaboost dengan 5 pengklasifikasi lemah
    clf.fit(X_train, y_train)  # Latih model
    y_pred = clf.predict(X_test)  # Prediksi set pengujian

    accuracy = accuracy_score(y_test, y_pred)  # Hitung akurasi
    print("Accuracy:", accuracy)  # Cetak akurasi



Accuracy: 0.39823008849557523
